# **Семинар 6 — Реализация стратегий развертывания и интеграция с облаком**

## **Цель занятия**

Научиться реализовывать стратегии безопасного развертывания ML-моделей (**blue-green** и **canary**)  
и автоматизировать публикацию контейнера в облаке с помощью CI/CD.

После семинара вы сможете:
- различать стратегии деплоя и выбирать оптимальную для ML-сервисов;
- использовать **Docker** и **GitHub Actions** для развёртывания контейнеров;
- подключать облачные сервисы (Render, AWS, Yandex Cloud и др.);
- проверять версию модели и состояние деплоя через `/health`;
- выполнять rollbacks и анализировать обновления моделей.


## **План занятия**

1. Повторение принципов Continuous Delivery и Deployment.  
2. Знакомство со стратегиями развертывания: **blue-green** и **canary**.  
3. Практическая реализация blue-green-деплоя в Docker.  
4. Пример canary-деплоя на уровне трафика.  
5. Интеграция GitHub Actions и публикация контейнера в облаке.  
6. Проверка `/health` и `/predict` после деплоя.  
7. Мониторинг, rollback и уведомления.


## **1. Введение: зачем нужны стратегии развертывания**

В ML-моделях обновления происходят часто — каждая новая версия может быть немного лучше…  
или **хуже**.

Если новая модель ухудшает качество, она может повлиять на пользователей и привести к убыткам.  
Поэтому развёртывания должны быть **без простоев** и с возможностью **мгновенного отката**.

### Основные стратегии:

| Стратегия | Описание | Применение |
|------------|-----------|-------------|
| **Blue-Green Deployment** | Две версии (blue — старая, green — новая) работают параллельно. Трафик переключается между ними. | Безопасное мгновенное переключение |
| **Canary Deployment** | Новая версия получает небольшой процент трафика (5–10%). Если всё стабильно — доля растёт до 100%. | Контролируемое тестирование модели |

💡 **Интерактив:**  
Почему в ML это особенно важно?  
→ Потому что **качество модели нельзя гарантировать заранее** — оно может упасть из-за смещения данных или ошибок препроцессинга.


## **2. Практика: реализация Blue-Green в Docker**

Создадим минимальный ML-сервис на **FastAPI** с версией модели, передаваемой через переменные окружения.


In [ ]:
# app/main.py
from fastapi import FastAPI
import os

app = FastAPI()
VERSION = os.getenv("MODEL_VERSION", "v1.0.0")

@app.get("/health")
def health():
    return {"status": "ok", "version": VERSION}

@app.get("/predict")
def predict():
    return {"prediction": "ok", "version": VERSION}


## **Dockerfile**

```Dockerfile
FROM python:3.11-slim
WORKDIR /app
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt
COPY . .
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8080"]



---

## **docker-compose.blue.yml**

```yaml
services:
  mlservice_blue:
    build: .
    environment:
      - MODEL_VERSION=v1.0.0
    ports:
      - "8080:8080"



## **docker-compose.green.yml**


```yaml
services:
  mlservice_green:
    build: .
    environment:
      - MODEL_VERSION=v1.1.0
    ports:
      - "8081:8080"



---

### **Запуск и проверка:**

```bash
docker compose -f docker-compose.blue.yml up -d
docker compose -f docker-compose.green.yml up -d



---

## **4. Canary Deployment: постепенное переключение трафика**

Вместо мгновенного переключения (Blue-Green),  
Canary-деплой распределяет трафик **частично** — например, 90% на старую модель, 10% на новую.

### Пример Nginx-конфигурации:

```nginx
http {
  upstream ml_backend {
    server blue.example.com weight=9;
    server green.example.com weight=1;
  }

  server {
    listen 80;
    location / {
      proxy_pass http://ml_backend;
    }
  }
}



---

## **5. Интеграция с GitHub Actions и облаком**

Теперь автоматизируем деплой через CI/CD.

Файл `.github/workflows/deploy.yml`:

```yaml
name: Deploy Model to Cloud

on:
  push:
    branches: [ main ]

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout code
        uses: actions/checkout@v4

      - name: Build Docker image
        run: docker build -t ghcr.io/${{ github.repository }}:${{ github.sha }} .

      - name: Push image to registry
        run: |
          echo "${{ secrets.GITHUB_TOKEN }}" | docker login ghcr.io -u ${{ github.actor }} --password-stdin
          docker push ghcr.io/${{ github.repository }}:${{ github.sha }}

      - name: Deploy to Cloud
        run: |
          curl -X POST https://api.cloudprovider/deploy \
            -H "Authorization: Bearer ${{ secrets.CLOUD_TOKEN }}" \
            -d "image=ghcr.io/${{ github.repository }}:${{ github.sha }}"



---

💡 **Интерактив**

Что нужно сделать перед использованием этого workflow?

Настроить **secrets.CLOUD_TOKEN**  
и дать Actions разрешение на доступ к GitHub Container Registry.


## **6. Проверка деплоя**

После успешного деплоя выполните:

```bash
curl https://<your-cloud-app>/health



---

```markdown
💡 **Вопрос на размышление**

Что произойдёт, если новая модель начнёт выдавать ошибки?

➡ Система должна выполнить **rollback** —  
автоматически вернуть трафик на старую версию (`blue`) или откатить контейнер к предыдущему тегу в реестре.


## **7. Мониторинг и Rollback**

Для production важно иметь:
- **метрики ошибок** (HTTP 5xx, latency, accuracy drift);
- **автоматический rollback** при превышении порога ошибок;
- **уведомления** о деплое (Slack, Telegram, Email).

Пример интеграции Slack:

```yaml
- name: Notify Slack
  uses: slackapi/slack-github-action@v2
  with:
    payload: '{"text":"✅ Model v1.1.0 deployed successfully"}'
  env:
    SLACK_WEBHOOK_URL: ${{ secrets.SLACK_WEBHOOK }}
